# Un simple vistazo

La idea es construir un módulo de funciones y publicarlo en <https://www.powershellgallery.com/> y <https://github.com/victortizs/ps-stuff>.

Ambos son lugares públicos de los que uno puede bajar (descargar/copiar/clonar) desde código en pedacitos (scripts), programas (software con uso propio, compatibles con otros programas o apps, o usados para hacer otros programas o apps) o apps (software con parte gráfica —GUI (<i>Graphical User Interface</i>)—. Se pueden bajar de varias formas: installers (.exe o .msi), apps portables (un zip generalmente) o bajar el código fuente —<i>source code</i>—, modificar, revisar y/o alterar si se quiere, y construir los ejecutables (los llaman <i>binaries</i> y son los .exe detrás de los 
íconos —<i>shortcuts</i> con extensión .lnk— que generalmente vemos en el escritorio).

**No pienso terminarlo todo para publicarlo** (no "se termina"), sino ir actualizándolo con el tiempo: agregando, modificando o quizá sacando cosas.

P. D. Además de las funciones debajo, me gustaría publicar en mi blog y/o mi cuenta de GitHub:
1. unos tips sobre la reducción de consumo de recursos (RAM y CPU), tanto dentro de apps, como modificando algunos registros o servicios del sistema;
2. recomendaciones de aplicaciones de código abierto o semi-abierto, buenas, bonitas y baratas para el sistema, en especial si se trata de computadores <i>low-end</i> (de poca capacidad de hardware o software algo viejo, sobre todo el hardware);
3. un ejemplo de como tratar con la API (<i>Application Programming Interface</i>, el mensajero/intermediario con apps u otros softwares) del BCRA (Banco Central de la República Argentina).

## Funciones

### Varias

<details>
    <summary>para agregar prefijos numéricos a carpetas recursivamente (necesita pulido)</summary>

```powershell
function Add-PrefixDirs {
    <#
    .SYNOPSIS
    Add a prefix with the format "[int]_" to all folders within the given path (root directory). Each level starts with i=1.
    .DESCRIPTION
    The function:
    (1) asks for a root directory (parent folder) to the user;
    (2) defines an interator ($i=1) for the root dir;
    (3) gets the parent folder's subfolders and call itself again for each of these, passing them as parent folders as well, setting an iterator $i=1 for them, and getting their respective subfolders;
    (4) renames all the defined parent folders' subfolders adding the prefix "[$i_]" to their names.
    .PARAMETER FOLDER
    The first and main root dir (parent folder), i.e., where you want to add a prefix to all subfolders recursively by level. It's your current location (working directory) by default.
    .EXAMPLE
    Add-PrefixDirs .\example\folder
    .EXAMPLE
    Add-PrefixDirs
    .EXAMPLE
    Add-PrefixDirs -Folder example\folder
    .INPUTS
    System.String
    .OUTPUTS
    System.Object
    .NOTES
    Helpful content on:
    (1) https://stackoverflow.com/questions/78197645/how-can-i-reset-a-counter-within-each-subfolder-level-in-a-loop-that-access-recu
    (2) https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.management/rename-item?view=powershell-7.4
    .LINK
    https://estudianteporahora.blog
    #>
    
    [CmdletBinding( 
        SupportsShouldProcess = $true,
        PositionalBinding = $true,
        ConfirmImpact = "Medium"
    )]

    Param(
        [Parameter(
            Mandatory = $false,
            HelpMessage = "Type the path to the parent folder where you want to applicate the function"
        )]
        [String]$Folder = ".\"
    )

    if ($PSCmdlet.ShouldProcess($Folder)) {
        $i = 1
        Get-ChildItem $Folder | Where-Object {$_.PSIsContainer -eq $true} | ForEach-Object {
            Add-PrefixDirs $_.FullName
            Rename-Item -Path $_.FullName -NewName ".\$($i)_$($_.Name)"
            $i++
        }
    }
}

New-Alias -Name predirs -Value Add-PrefixDirs
```
</details>

<details>
    <summary>para buscar archivos y/o carpetas dentro de un directorio, numerándolos</summary>

```powershell
function Measure-DiFi {
    <#
    .SYNOPSIS
    Measure the number of folders and/or files inside a given root directory.
    .DESCRIPTION
    The function:
    (1) asks for a path to a root dir or parent folder;
    (2) prompts you to choose if you want to search for files, folders, or both of them;
    (3) prompts you to choose if you want to show the output structure or display each name.
    .PARAMETER FOLDER
    Path to the folder/dir you want to analyse. Uses your current location '.\' by default.
    .PARAMETER SEARCH
    Accepts 'dirs', 'files', or 'both' as input. Looks for 'both' by default.
    .PARAMETER DISPLAY
    Accepts 'structure' or 'names' as input. Show 'names' by default.
    .EXAMPLE
    Measure-DiFi -Search Dirs -Display Names
    .EXAMPLE
    Measure-DiFi .\example\folder -Display Structure
    .EXAMPLE
    Measure-Difi
    .INPUTS
    System.String
    .OUTPUTS
    System.Object
    .NOTES
    Helpful content on:
    (1) https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.utility/measure-object?view=powershell-7.4
    .LINK
    https://estudianteporahora.blog
    #>

    [CmdletBinding(
        SupportsShouldProcess = $true
    )]

    Param(
        [Parameter(
            Mandatory = $false,
            ValueFromPipeline = $true,
            ValueFromPipelineByPropertyName = $true,
            HelpMessage = "Path to the folder/drive that you want to explore recursively."
        )]
        [String]$Folder = ".\",
        [Parameter(
            Mandatory = $false,
            HelpMessage = "Type 'dirs' to look only for dirs, 'files' to look only for files, or 'both' to look for both of them."
        )]
        [ValidateSet("Dirs", "Files", "Both")]
        [String]$Search = "Both",
        [Parameter(
            Mandatory = $false,
            HelpMessage = "Type 'structure' to choose a structured output or 'names' to choose a list-formatted output."
        )]
        [ValidateSet("Structure", "Names")]
        [String]$Display = "Names"
    )

    if ($PSCmdlet.ShouldProcess($Folder)) {
        $Directories = Get-ChildItem $Folder -Directory -Recurse
        $Files = Get-ChildItem $Folder -File -Recurse
        Switch ($Search) {
            # Search just for directories/folders
            {$_ -eq "Dirs"} {
                if (($Directories | Measure-Object).Count -eq 0) {
                    Write-Host "`nThe current item on this path '$(Get-Item $Folder)' has no subfolders within.`n"
                }
                else {
                    Write-Host "`nThe parent folder '$(Get-item $Folder)' has a total of $(($Directories | Measure-Object).Count) subfolder(s) within:`n"
                    Switch ($Display) {
                        {$_ -eq "Names"} {
                            $i = 1
                            $Directories | ForEach-Object {
                                Write-Host ("{0}. {1}" -f $i, $_.FullName)
                                $i++
                            }
                            Write-Host ""
                        }
                        {$_ -eq "Structure"} {
                            Write-Host "The folder $(Get-Item $Folder) has $((Get-ChildItem $Folder -Directory | Measure-Object).Count) subfolder(s)."
                            # Descend structure
                            $Directories | ForEach-Object {
                                if ((Get-ChildItem $_.FullName -Directory | Measure-Object).Count -gt 0) {
                                    Write-Host "The folder $($_.FullName) has $((Get-ChildItem $_.FullName -Directory | Measure-Object).Count) subfolder(s)."
                                }
                            }
                            Write-Host ""
                        }
                    }
                }
            }
            # Search just for files
            {$_ -eq "Files"} {
                if (($Files | Measure-Object).Count -eq 0) {
                    Write-Host "`nThe current item on this path '$(Get-Item $Folder)' has no files within.`n"
                }
                else {
                    Write-Host "`nThe parent folder '$(Get-Item $Folder)' has a total of $(($Files | Measure-Object).Count) file(s) within:`n"
                    Switch ($Display) {
                        {$_ -eq "Names"} {
                            $i = 1
                            $Files | ForEach-Object {
                                Write-Host ("{0}. {1}" -f $i, $_.FullName)
                                $i++
                            }
                            Write-Host ""
                        }
                        {$_ -eq "Structure"} {
                            if ((Get-ChildItem $Folder -File | Measure-Object).Count -gt 0) {
                                Write-Host "The folder $(Get-Item $Folder) has $((Get-ChildItem $Folder -File | Measure-Object).Count) file(s)."
                            }
                            # Descend structure
                            $Directories | ForEach-Object {
                                if ((Get-ChildItem $_.FullName -File | Measure-Object).Count -gt 0) {
                                    Write-Host "The folder $($_.FullName) has $((Get-ChildItem $_.FullName -File | Measure-Object).Count) file(s)."
                                }  
                            }
                            Write-Host ""
                        }
                    }
                }
            }   
            # Search for both directories and files
            {$_ -eq "Both"} {
                # First case: parent folder contains both dirs and files
                if (($Directories | Measure-Object).Count -gt 0 -and ($Files | Measure-Object).Count -gt 0) {
                    # Show dir results for parent folder
                    Write-Host "`nThe parent folder '$(Get-Item $Folder)' has a total of $(($Directories | Measure-Object).Count) subfolder(s) within:`n"
                    Switch ($Display) {
                        {$_ -eq "Names"} {
                            $i = 1
                            $Directories | ForEach-Object {
                                Write-Host ("{0}. {1}" -f $i, $_.FullName)
                                $i++
                            }
                            Write-Host ""
                        }
                        {$_ -eq "Structure"} {
                            Write-Host "The folder $(Get-Item $Folder) has $((Get-ChildItem $Folder -Directory | Measure-Object).Count) subfolder(s)."
                            # Descend structure and show results
                            $Directories | ForEach-Object {
                                if ((Get-ChildItem $_.FullName -Directory | Measure-Object).Count -gt 0) {
                                    Write-Host "The folder $($_.FullName) has $((Get-ChildItem $_.FullName -Directory | Measure-Object).Count) subfolder(s)."
                                }
                            }
                            Write-Host ""
                        }
                    }
                    # Show file results for parent folder
                    Write-Host "The parent folder '$(Get-Item $Folder)' has a total of $(($Files | Measure-Object).Count) file(s) within:`n"
                    Switch ($Display) {
                        {$_ -eq "Names"} {
                            $i = 1
                            $Files | ForEach-Object {
                                Write-Host ("{0}. {1}" -f $i, $_.FullName)
                                $i++
                            }
                            Write-Host ""
                        }
                        {$_ -eq "Structure"} {
                            if ((Get-ChildItem $Folder -File | Measure-Object).Count -gt 0) {
                                Write-Host "The folder $(Get-Item $Folder) has $((Get-ChildItem $Folder -File | Measure-Object).Count) file(s)."
                            }
                            # Descend structure and show results
                            $Directories | ForEach-Object {
                                if ((Get-ChildItem $_.FullName -File | Measure-Object).Count -gt 0) {
                                    Write-Host "The folder $($_.FullName) has $((Get-ChildItem $_.FullName -File | Measure-Object).Count) file(s)."
                                }
                            }
                            Write-Host ""                          
                        }
                    }
                }
                # Second case: parent folder only has files
                if (($Directories | Measure-Object).Count -eq 0 -and ($Files | Measure-Object).Count -gt 0) {
                    Write-Host "`nThe current item on this path '$(Get-Item $Folder)' has no subfolders within.`n"
                    Write-Host "The parent folder '$(Get-Item $Folder)' has a total of $(($Files | Measure-Object).Count) file(s) within:`n"
                    Switch ($Display) {
                        {$_ -eq "Names"} {
                            $i = 1
                            $Files | ForEach-Object {
                                Write-Host ("{0}. {1}" -f $i, $_.FullName)
                                $i++
                            }
                            Write-Host ""
                        }
                        {$_ -eq "Structure"} {
                            Write-Host "The folder $(Get-Item $Folder) has $((Get-ChildItem $Folder -File | Measure-Object).Count) file(s).`n"
                        }
                    }
                }
                # Third case: parent folder only has dirs
                if (($Directories | Measure-Object).Count -gt 0 -and ($Files | Measure-Object).Count -eq 0) {
                    Write-Host "`nThe current item on this path '$(Get-Item $Folder)' has no files within.`n"
                    Write-Host "The parent folder '$(Get-Item $Folder)' has a total of $(($Directories | Measure-Object).Count) subfolder(s) within:`n"
                    Switch ($Display) {
                        {$_ -eq "Names"} {
                            $i = 1
                            $Directories | ForEach-Object {
                                Write-Host ("{0}. {1}" -f $i, $_.FullName)
                                $i++
                            }
                            Write-Host ""
                        }
                        {$_ -eq "Structure"} {
                            Write-Host "The folder $(Get-Item $Folder) has $((Get-ChildItem $Folder -Directory | Measure-Object).Count) subfolder(s)."
                            # Descend structure
                            $Directories | ForEach-Object {
                                if ((Get-ChildItem $_.FullName -Directory | Measure-Object).Count -gt 0) {
                                    Write-Host "The folder $($_.FullName) has $((Get-ChildItem $_.FullName -Directory | Measure-Object).Count) subfolder(s)."
                                }             
                            }
                            Write-Host ""
                        }
                    }
                }
                # Fourth and last case: parent folder doesn't have any items within
                if (($Directories | Measure-Object).Count -eq 0 -and ($Files | Measure-Object).Count -eq 0) {
                    Write-Host "`nThe current item on this path '$(Get-Item $Folder)' has no content.`n"
                }
            }
        }
    }
}

New-Alias -Name difi -Value Measure-DiFi
```
</details>

<details>
    <summary>para buscar en la web eligiendo el navegador desde la terminal, rápidamente (en un futuro se me ocurre que también devuelva los resultados en la terminal y no en una pestaña del navegador predeterminado</summary>

```powershell
function Pop-Web {
    <#
    .SYNOPSIS
    Browse the web from the shell.
    .DESCRIPTION
    The function uses user input to launch a specific browser (duck, google or bing) with a given query. You can later create an alias in your PS profile for this function, like "web".
    .PARAMETER B
    The browser. if you omit this parameter it will use duck by default. Also accepts pipeline input.
    .EXAMPLE
    Pop-Web -B google
    .EXAMPLE
    Pop-Web g
    .EXAMPLE
    $Browser = "bing"
    Pop-Web $Browser
    .INPUTS
    System.String
    .OUTPUTS
    System.Diagnostics.Process
    .NOTES
    Helpful content on:
    (1) https://adamtheautomator.com/powershell-modules/#Installing_Modules
    (2) https://mikefrobbins.com/2013/07/04/how-to-create-powershell-script-modules-and-module-manifests/

    Also remember you can create an alias in your PS profile for this function, to make it smoother.
    .LINK
    https://estudianteporahora.blog
    #>

    [CmdletBinding (
        SupportsShouldProcess = $true
    )]

    Param (
        [Parameter (
        Mandatory = $false,
        ValueFromPipeline =$true,
        ValueFromPipelineByPropertyName = $true,
        HelpMessage = "Please choose a browser between duck (duckduckgo), google, and bing. This parameter is not mandatory, the default value is duck."
        )]
        [ValidateSet("google", "g", "bing", "b", "duck", "d")]
        [String]$B = "duck"
    )

    $Query = Read-Host ("Write your query")

    if ($PSCmdlet.ShouldProcess($B)) {
        Switch ($B) {
        {$_ -eq "google" -or $_ -eq "g"} {Start-Process ("https://google.com/search?q=" + $Query -Replace ("-","+"))}
        {$_ -eq "bing" -or $_ -eq "b"} {Start-Process ("https://bing.com/search?q=" + $Query -Replace ("-", "+"))}
        {$_ -eq "duck" -or $_ -eq "d"} {Start-Process ("https://duckduckgo.com/?q=" + $Query -Replace ("-", "+"))}
        }
    }
}

New-Alias -Name web -Value Pop-Web
```
</details>

<details>
    <summary>para fijar un temporizador en la terminal con dos teclas para pausar y resumir y tiempo restante reflejado en la terminal (emite un sonido al final para avisar que terminó)</summary>

```powershell
function Set-Timer {
    [CmdletBinding(
        SupportsShouldProcess = $true,
        PositionalBinding = $true,
        ConfirmImpact = "Low"
    )]

    Param (
        [Parameter(
            Mandatory = $true,
            HelpMessage = "Type the minutes you wish your timer to be set."
        )]
        [Double]$Minutes
    )

    if ($PSCmdlet.ShouldProcess($Minutes)) {

        Write-Host "`nPress 'P' or 'p' to pause the timer and 'R' or 'r' to resume it.`n"
        $Paused = $false # Flag to track pause state
        $TotalSec = $Minutes * 60

        for ($i = $TotalSec; $i -gt 0; $i--) {

            # Capture keyboard input asynchronously
            if ([System.Console]::KeyAvailable) {
                $Key = [System.Console]::ReadKey($true).Key
                if ($Key -eq 'P'-and -not $Paused) {
                    $Paused = $true
                    Write-Host "`nTimer paused. Press 'R' to resume..."
                }
            }

            # Pause countdown if the paused flag is set to true and read next key to resume
            while ($Paused) {
                Start-Sleep -Milliseconds 200 # Small delay to prevent high CPU usage
                if ([System.Console]::KeyAvailable) {
                    $Key = [System.Console]::ReadKey($true).Key
                    if ($Key -eq 'R') {
                        $Paused = $false
                        Write-Host "`nTimer resumed."
                    }
                }
            }

            # Calculations
            $HrsDisplayed = ([System.Math]::Floor($i / 3600)).ToString("00")
            $MinDisplayed = ([System.Math]::Floor(($i % 3600) / 60).ToString("00"))
            $SecDisplayed = ($i % 60).ToString("00")

            Write-Host "`rTime left: $HrsDisplayed h $MinDisplayed min $SecDisplayed s." -NoNewline
            Start-Sleep -Seconds 1
        }

        Write-Host "`nDone! Go rest for some minutes.`n"
        [System.Console]::Beep(500, 1750)
    }
}

New-Alias -Name timer -Value Set-Timer

```
</details>

<details>
    <summary>para manejar y ver dentro de la papelera de reciclaje —<i>recycle bin</i>— desde la terminal de manera fácil (necesito completar un parámetro, pero ya es utilizable)</summary>

```powershell
function Start-RecycleBin {
    [CmdletBinding(
        SupportsShouldProcess = $true,
        PositionalBinding = $true
    )]

    Param(
        [Parameter(
            Mandatory = $false,
            HelpMessage = "Action to perform, either open the bin in the file explorer, list the items on the terminal, delete them or restore them."
        )]
        [ValidateSet("Open", "List", "Restore-ByName", "Restore-All", "Remove-ByName", "Remove-ByAll")]
        [String]$Action = "Open",

        [Parameter(
            Mandatory = $false,
            HelpMessage = "Name of the file, folder, etc., to provide after Remove-ByName parameter."
        )]
        [String[]]$Name,

        [Parameter(
            Mandatory = $false,
            HelpMessage = "Location to provide in case of duplicated file names."
        )]
        [string[]]$OgLocation
    )

    $Shell = New-Object -ComObject Shell.Application
    $RecycleBin = $Shell.NameSpace(0xA) # 0xA = Recycle Bin
    $DynamicItems = $RecycleBin.Items() # $RecycleBin.Items() is live and mutable (live COM object), may causing skipping in some altering loops

    $StaticItems = @() # Snapshot copy, won't change in loop
    for ($i = 0; $i -lt $DynamicItems.Count; $i++) {
        $StaticItems += $DynamicItems.Item($i)
    }


    if ($PSCmdlet.ShouldProcess($Action)){
        Switch ($Action) {
            {$_ -eq "Open"} {
                explorer.exe shell:RecycleBinFolder
            }
            {$_ -eq "List"} {
                $Count = $DynamicItems.Count
                if ($Count -eq 0) {
                    Write-Host "`nNo data here.`n"
                }
                else {
                    $i = 1
                    Write-Host "`n$Count object(s) here."
                    $DynamicItems | ForEach-Object {
                        [PSCustomObject]@{
                            "ItemNo" = $i++
                            "Name" = $_.Name
                            "Size(KB)" = [int]([Math]::Round($_.Size / 1024, 0))
                            "Type" = $_.Type
                            "OrigLoc" = $RecycleBin.GetDetailsOf($_, 1) # 1 = Original Location, 2 = Date Deleted
                            "DeletedOn" = $RecycleBin.GetDetailsOf($_, 2) -Replace "[^\u0000-\u007F]", ""
                        }
                    }
                }
            }

            {$_ -eq "Restore-ByName"} {
                if (-not $Name) {
                    Write-Host "`nPlease provide a name using the -Name parameter`n" -ForegroundColor Red
                    Return
                }

                if (-not $OgLocation) {
                    Write-Host "if you have two or more items named the same, coming from different locations, you won't be able to specified which one to restore unless you use the -OgLocation parameter.`n"
                    Write-Host "In other words, if you have two files named the same, and did not use the -OgLocation parameter, both files will be restored without any confirmation prompt.`n"
                }

                Write-Host "Note 1: Windows only asks for confirmation when restoring individual items if there's another item with the same name in their original location." -ForegroundColor Yellow

                $FilteredItems = $StaticItems | Where-Object { 
                    $Name -contains $_.Name -and 
                    (
                        (-not $OgLocation) -or
                        ($OgLocation -contains $RecycleBin.GetDetailsOf($_, 1))
                    )
                }

                $Restored = 0
                $FilteredItems | ForEach-Object {
                    $Item = $_
                    $Verbs = $Item.Verbs()
                    foreach ($Verb in $Verbs) {
                        if ($Verb.Name.Replace("&", "") -match "^(Restore|Restaurar)$") {
                            $NameToMatch = $Item.Name
                            $OrigLoc = $RecycleBin.GetDetailsOf($_, 1)

                            # Attempt deletion
                            $Verb.DoIt()
                            Start-Sleep -Milliseconds 500 # Allows UI time to process the deletion
                            
                            # Refresh snapshot
                            $DynamicItems = $RecycleBin.Items()
                            $AfterItems = @()
                            for ($j = 0; $j -lt $DynamicItems.Count; $j++) {
                                $AfterItems += $DynamicItems.Item($j)
                            }

                            # Check if item is gone for good
                            $Existence = $AfterItems | Where-Object {
                                $_.Name -eq $NameToMatch -and
                                $RecycleBin.GetDetailsOf($_, 1) -eq $OrigLoc
                            }

                            if (-not $Existence) {
                                $Restored++
                            }
                            Break                            
                        }
                    }
                }

                if ($Restored -gt 0) {
                    Write-host "`n$Restored item(s) restored from Recycle Bin.`n" -ForegroundColor Green
                } else {
                    Write-Host "`nNo items restored. Verb 'Restore' not found or action canceled by the user.`n" -ForegroundColor Red
                }

                <#
                Chat GPT:
                This code does the job, but when I have 2 items named the same, from different original locations, and I use the parameter -Name with their name, I want to be able to choose which one to restore, cuz the function restores both of them, without any confirmation through a dialogue box, as opposed when deleting by name. Any possible suggestion? I think of adding another parameter named "OrigLoc", but I also believe it can be done in a more efficient way
                #>

            }               

            {$_ -eq "Restore-All"} {
                $Restored = 0
                $StaticItems | ForEach-Object {
                    $Item = $_
                    $Verbs = $Item.Verbs()
                    foreach ($Verb in $Verbs) {  # foreach behaves distinctively to For-EachObject, it's quicker and gives better flow control 
                        if ($Verb.Name.Replace("&", "") -match "^(Restore|Restaurar)$"){ # Also works with -eq "delete"
                            $Verb.DoIt()
                            $Restored++
                            Break
                        }
                    }
                }

                if ($Restored -gt 0) {
                    Write-Host "`n$Restored item(s) restored from Recycle Bin.`n" -ForegroundColor Green
                } else {
                    Write-Host "`nNo items restored. Verb 'Restore' not found?`n" -ForegroundColor Red
                }
            }

            {$_ -eq "Remove-ByName"} {
                <#
                $Items | % {
                    $RawDate = $RecycleBin.GetDetailsOf($_, 2)
                    
                    # Clean any invisible characters (like formatting ones that break parsing, comparison and sorting; plus they're invisible)
                    # They often come from Windows Shell COM output, especially dates and file paths

                    $CleanDate = $RawDate -replace "[^\u0000-\u007F]", ""  # Remove RTL markers, etc. Also works $RawDate -replace '[\u200E\u200F\u202A-\u202E\u2066-\u2069\uFEFF]', ''
                    # The previous regex removes any character not in the ASCII range, which includes:
                    # - RTL markers (U+200E, U+200F), they control text direction
                    # - Non-breaking spaces
                    # - Most Unicode symbols
                    # - Accented or localized characters

                    # Parsing to a datetime object PowerShell style
                    Try {
                        $DateDeleted = [datetime]$CleanDate # Also [datetime]::Parse($CleanDate)
                        $FormattedDate = $DateDeleted.ToString("yyyyMMdd_HHmmss")
                    } Catch {
                        $FormattedDate = "invalid date" # In case the parsing fails go back to the raw output
                    }

                    [PSCustomObject]@{
                    "Name" = $_.Name
                    "OrigLoc" = $RecycleBin.GetDetailsOf($_, 1) # 1 for original location, 2 for date of deletion
                    "DateDeleted" = $FormattedDate
                    }
                }
                #>

                <#
                # Here is noticeable that the only characters which do not belong to ASCII are 8206 (U+200E) and 8207 (U+200F)
                $Items | % {
                    $Temp1 = $RecycleBin.GetDetailsOf($_, 2)
                    foreach ($Temp in $Temp1) {
                        "$($Temp.ToCharArray() | % { [int][char]$_ })" # or "$($Temp.ToCharArray() | % { "{0} = u{1:X4}," -f [int][char]$_ })"
                        Break
                    }
                }                                
                #>

                if (-not $Name) {
                    Write-Host "`nPlease provide a name using the -Name parameter`n" -ForegroundColor Red
                    Return
                }
                
                $Removed = 0

                Write-Host "`nNote 1: Windows may prompt for confirmation when deleting individual items." -ForegroundColor Yellow
                Write-Host "`nNote 2: Also consider that if you have two or more items named the same, coming from different locations, you can choose which one to delete in the dialogue box, just pay attention to its 'Original Location'." -ForegroundColor Yellow
                $StaticItems | Where-Object { $Name -contains $_.Name } | ForEach-Object {
                    $Item = $_
                    $Verbs = $Item.Verbs()
                    foreach ($Verb in $Verbs) {
                        if ($Verb.Name.Replace("&", "") -match "^(Delete|Eliminar)$") {
                            $NameToMatch = $Item.Name
                            $OrigLoc = $RecycleBin.GetDetailsOf($Item, 1)

                            # Attempt deletion
                            $Verb.DoIt()
                            Start-Sleep -Milliseconds 500 # Allows UI time to process the deletion

                            # Refresh snapshot
                            $DynamicItems = $RecycleBin.Items()
                            $AfterItems = @()
                            for ($j = 0; $j -lt $DynamicItems.Count; $j++) {
                                $AfterItems += $DynamicItems.Item($j)
                            }

                            # Check if item is gone for good
                            $Existence = $AfterItems | Where-Object {
                                $_.Name -eq $NameToMatch -and 
                                $RecycleBin.GetDetailsOf($_, 1) -eq $OrigLoc
                            }

                            if (-not $Existence) {
                                $Removed++
                            }
                            Break
                        }
                    }
                }         

                if ($Removed -gt 0) {
                    Write-host "`n$Removed item(s) removed from Recycle Bin.`n" -ForegroundColor Green
                } else {
                    Write-Host "`nNo items removed. Verb 'Delete' not found or action canceled by the user.`n" -ForegroundColor Red
                }                

            }
        }
    }
}

New-Alias -Name recbin -Value Start-RecycleBin
```
</details>

### Que son atajos a programas o aplicaciones

<details>
    <summary>para abrir aplicaciones instaladas por default en Windows desde la terminal. La idea de hacerlo desde la terminal es evitar distracciones y trabajar o usar una sola herramienta para cosas generales</summary>

```powershell
function Start-ToDo {
    explorer.exe shell:appsfolder\Microsoft.todos_8wekyb3d8bbwe!App
}

New-Alias -Name tasks -Value Start-ToDo

function Start-Weather {
    explorer.exe shell:appsfolder\Microsoft.BingWeather_8wekyb3d8bbwe!App
}

New-Alias -Name weather -Value Start-Weather
```
</details>

<details>
    <summary>para otras aplicaciones o programas "no nativos" de Windows (estas probablemente no vayan en la publicación)</summary>

```powershell
function Find-Everything {
    & "$HOME\Tools\PortableApps\Everything-1.4.1.1032.x64\es.exe" -dm -sort dm -highlight @args
}

New-Alias -Name f -Value Find-Everything    

function Start-GitBash { 
    Start-Process "C:\Program Files\Git\git-bash.exe" 
}

New-Alias -Name gbash -Value Start-GitBash
```
</details>

### Que necesitan una o dos manos más para cumplir su propósito

<details>
    <summary>para actualizaciones de Windows (salen el segundo martes de cada mes —<i>Patch Tuesday</i>—) no temporales —<i>Previews</i>—. Instala el módulo PSWindowsUpdate para eso, sino lo encuentra instalado, y define un registro —<i>log</i>— para guardar la info. Quizá la convierta en un script (tengo algunos, que no son funciones como tal y no se incluyen en un módulo, son de uso más personal)</summary>

```powershell
function Update-Windows {
    [CmdletBinding(
        SupportsShouldProcess = $true,
        PositionalBinding = $true,
        ConfirmImpact = "Medium"
    )]

    Param(
        [Parameter(
            Mandatory = $false,
            HelpMessage = "Path to the log file container where you want Windows updates to be recorded. Default is 'C:\Users\YourUser\Scripts\Logs\"
        )]
        [String]$LogPath = "Default",
        [Parameter(
            Mandatory = $false,
            HelpMessage = "Name you want the file (.txt) to have. Default is 'WU_Log'"
        )]
        [string]$Filename = "WU_Log"
    )

    $VerbosePreference = "Continue"
    $TimeStamp = Get-Date -Format "yyyyMMdd_HHmmss"

    if ($PSCmdlet.ShouldProcess($LogPath)){
        if (Get-Module -ListAvailable -Name PSWindowsUpdate) {
            # mark
            if ($LogPath -ne "default") { 
                if (Test-Path -Path $LogPath) {
                    Write-Host "No need to create a log container. Proceeding..."
                }
                else {
                    New-Item -Path $LogPath -ItemType Directory -Verbose
                }
            }
            else {
                if (Test-Path -Path "$($home)\Scripts\Logs") {
                    Start-Sleep -Seconds 2
                } 
                else {
                    $LogPath = New-Item -Path "$($home)\Scripts\Logs" -ItemType Directory -Verbose
                    Write-Verbose -Message "Remember your output will be at '$($home)\Scripts\Logs'" -Verbose
                }
            }
            # mark
            # Start-Transcript -Path "$($LogPath)\$($Filename)_$($TimeStamp).txt" -NoClobber
            # Install-WindowsUpdate -AcceptAll -AutoReboot -Verbose
            Get-WindowsUpdate | Tee-Object -FilePath "$($LogPath)\$($Filename)_$($TimeStamp).txt" -Append
            # Stop-Transcript
            Write-Verbose -Message "Done. Come back soon!" -Verbose
        }
        else {
            Write-Warning "`nRequired module does not exist.`nInstalling module PSWindowsUpdate...`nWhen prompted, select [A] to proceed with the installation properly.`n...`n"
            # ask the user if they want to proceed or not and adapt the code based on it
            Install-Module -Name PSWindowsUpdate -Scope CurrentUser
            Write-Host ""
            # mark
            # can I summarize/define this block used twice in the code?
            if ($LogPath -ne "default") { 
                if (Test-Path -Path $LogPath) {
                    Write-Host "No need to create a log container. Proceeding..."
                }
                else {
                    New-Item -Path $LogPath -ItemType Directory -Verbose
                }
            }
            else {
                if (Test-Path -Path "$($home)\Scripts\Logs") {
                    Start-Sleep -Seconds 2
                } 
                else {
                    $LogPath = New-Item -Path "$($home)\Scripts\Logs" -ItemType Directory -Verbose
                    Write-Verbose -Message "Remember your output will be at '$($home)\Scripts\Logs'" -Verbose
                }
            }
            #mark
            # Start-Transcript -Path "$($LogPath)\$($Filename)_$($TimeStamp).txt" -NoClobber
            # Install-WindowsUpdate -AcceptAll -AutoReboot -Verbose
            Get-WindowsUpdate | Tee-Object -FilePath "$($LogPath)\$($Filename)_$($TimeStamp).txt" -Append
            # Stop-Transcript
            Write-Verbose -Message "Done. Come back soon!" -Verbose
            ;
            Break
        }
    }
}

New-Alias -Name udwin -Value Update-Windows
``` 
</details>

<details>
    <summary>función para intercambiar las posiciones de elementos dentro de una serie —<i>array</i>—, es simple y funciona, pero quiero agregar una opción para que el usuario elija si sobreescribir la serie original o crear otra con los elementos en su nueva posición, y la terminar la descripción (luego de terminar lo primero)</summary>

```powershell
function Switch-ArrayElements {
    # .SYNOPSIS
    Swap elements of an array based on pairs of indices.
    # .DESCRIPTION
    The function asks for an array and pairs of indices (lists of integers of exactly two elements), each pair defines the elements that are gonna be swaped.
    # .PARAMETER array
    This parameter asks the user for a list (array) with any type of elements (string, list, boolean, double, etc.)
    # .PARAMETER pairs
    This parameter asks the user for a list (array) of two-element lists (arrays), which indicate positions to swap in the original array, each pair treated as one atomic swap, instead of parallel arrays.
    # .EXAMPLE
    Switch-ArrayElements -array $list -pairs @( @(0,2), @(1,3) )
    # .INPUTS
    System.Array
    # .OUTPUTS

    # .NOTES

    # .LINK

    param(
        [array]$array,
        [int[][]]$pairs
    )

    foreach ($pair in $pairs) {
        if ($pair.Count -ne 2) {
            throw "Each pair must contain only 2 elements to be passed as indices."
        }

        $i, $j = $pair
        ($array[$i], $array[$j]) = ($array[$j], $array[$i])
    }

    return $array
}

New-Alias -Name swindexes -Value Switch-ArrayElements
```
</details>

### Que hice en mi último laburo y voy a sumar

<details>
    <summary>para encriptar zips con una contraseña elegible o seleccionada por default, depende de otro módulo que instala si no encuentra instalado previamente y simplifica un poco su uso para ese propósito incluyendo algunos parámetros por default</summary>

```powershell
Function Crip_Zip
{
    [CmdletBinding(
        SupportsShouldprocess = $True
        )]

    Param (
        [Parameter (
            Mandatory = $True,
            ValueFromPipeline = $True,
            ValueFromPipelineByPropertyName = $True,
            HelpMessage = "Path to the folder you want to zip."
        )]
        [String] $Folder_Path,
        [Parameter (
            Mandatory = $True,
            ValueFromPipeline = $True,
            ValueFromPipelineByPropertyName = $True,
            HelpMessage = "Path to the folder where you want the zip to appear/be created."
        )]
        [String] $Zip_Path,
        [Parameter (
            Mandatory = $False,
            HelpMessage = "The password for the zip. If you omit this parameter it will use the one for Cisco RU by default."
        )]
        [String] $Zip_Key = "Frodo" # this is not the real default they use LOL
    )
    
    $Zip_Name = Read-Host('Type the name you want the zip to have')
    Write-Host ""
    $VerbosePreference = "Continue"

    if ($PSCmdlet.ShouldProcess($Folder_Path)) {
        if (Get-Module -ListAvailable -Name 7Zip4PowerShell) {
            $PassCode = ConvertTo-SecureString $Zip_Key -AsPlainText -Force
            Compress-7Zip -Path $Folder_Path -ArchiveFileName "$($Zip_Name).zip" -OutputPath $Zip_Path -Format Zip -SecurePassword $PassCode
            Write-Verbose -Message "Done. Come back soon!" -Verbose
            Write-Host ""
        } 
        else {
            Write-Warning "`nRequired module does not exist.`nInstalling module 7Zip4PowerShell...`nWhen prompted, select [A] to proceed with the installation properly.`n...`n"
            Install-Module -Name 7zip4PowerShell -Scope CurrentUser
            Write-Host ""
            $PassCode = ConvertTo-SecureString $Zip_Key -AsPlainText -Force
            Compress-7Zip -Path $Folder_Path -ArchiveFileName "$($Zip_Name).zip" -OutputPath $Zip_Path -Format Zip -SecurePassword $PassCode
            Write-Verbose -Message "Done. Come back soon!" -Verbose
            Write-Host ""
            ;
            Break
        }
    }
}
```
</details>

<details>
    <summary>para convertir el formato estándar de archivos de texto (.txt) a uno a elección o el predeterminado por la función</summary>

```powershell
Function UTF_16LE
{
    [CmdletBinding(
        SupportsShouldprocess = $True
    )]   

    Param (
        [Parameter (
            Mandatory = $True,
            ValueFromPipeline = $True,
            ValueFromPipelineByPropertyName = $True,
            HelpMessage = "Path to the folder containing the '.txt' files you wish to convert to UTF-16 LE or other encoding."
        )]
        [String] $Folder_Path,
        [Parameter (
            Mandatory = $False,
            ValueFromPipeline = $True,
            ValueFromPipelineByPropertyName = $True,
            HelpMessage = "The encoding you wish to apply to the file(s). E.g.: 'ascii', 'ansi', 'utf7', 'utf8', 'unicode', 'utf8NoBom', 'utf32'."
        )]
        [String] $Enc = "Unicode"
    )

    $VerbosePreference = "Continue"

    If ($PSCmdlet.ShouldProcess($Folder_Path)) {
        $Notice = Read-Host("`nIf you did not specified the encoding this function will convert your files to 'UTF-16 LE'(which is unicode).`nDo you want to proceed? Please answer with 'yes', 'y' or 'no', 'n'.")
        Switch ($Notice) {
            {$Notice -eq "yes" -or $Notice -eq "y"} {
                If ((Get-ChildItem $Folder_Path -File -Filter *.txt | Measure-Object).Count -eq 0) {
                    Write-Host ""
                    Write-Verbose -Message "There are no '.txt' files in the input path. Please be aware that this function does not access to the given path recursively."
                    Write-Host ""
                }
                Else {
                    Write-Host ""
```
</details>

### Bonus track

También hice un [Excel](https://docs.google.com/spreadsheets/d/1n1FgSfhSD3UoHd3b2l9FaVu6l1MYP3GB/edit?usp=drive_link&ouid=107951629212498104456&rtpof=true&sd=true "link al Excel guardado en Google Drive") que devuelve contenido para pegar en un archivo de texto, que se debe guardar como .bat, correr, y crea "x" cantidad de carpetas y subcarpetas con muy pocos inputs. Quizá suba el Excel o vea una alternativa que haga lo mismo con lenguajes como PowerShell o Python.